In [6]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *
import plotly.graph_objs as go
import numpy as np


In [9]:
def process_data(Probs_text, Outcomes_text, RN_text):
    Probs=np.fromstring(Probs_text.value, dtype=float, sep=',')
    CDF= np.append(0,np.cumsum(Probs))
    x=np.repeat(0,len(CDF))
    Outcomes=np.fromstring(Outcomes_text.value, dtype=float, sep=',')
    RN=np.fromstring(RN_text.value, dtype=float, sep=',')[0]
    return Probs, CDF,x,Outcomes,RN


 
# Assign an empty figure widget with two traces


def convert(Outcomes,CDF,RN):
    Noutcomes=len(Outcomes)
    for i in range(Noutcomes):
        if RN<=CDF[i+1]:
            return i



def initialize(): 
    Outcomes_text=widgets.Text(
        value="1,2,3",
        description='a value:',
        disabled=False
    )

    Probs_text=widgets.Text(
        value="0.3,0.4,0.3",
        description='a value:',
        disabled=False
    )


    RNG=widgets.Button(description="Generate RN")
    update=widgets.Button(description="new distribution")
    search_rn=widgets.Checkbox(
        value=False,
        description='Search',
        disabled=False,
        indent=False
    )


    RN_text=widgets.Text(
        value="0.1",
        description='a value:',
        disabled=False
    )

    FB=widgets.HTML(
        value="<b></b>",
        placeholder='Some HTML',
        description='',
    )
 
    Probs,CDF,x,Outcomes,RN=process_data(Probs_text, Outcomes_text, RN_text)
 
    i=convert(Outcomes,CDF,RN) 
    
    trace1=go.Bar(x=x,y=Probs, marker=dict(color=CDF,colorscale="PuBu"), 
                  hoverinfo="text",text=Outcomes,name="Outcome regions"+' '*20   )
    trace2=go.Scatter(x=[0],y=[RN],hoverinfo="text",text="u="+str(RN)+"<br>converts to "+str(Outcomes[i]),
                      marker=dict(size=20,color="red"),name="Random number")
    trace3=go.Scatter(x=x,y=CDF,mode='markers+lines+text', hoverinfo="skip", text=np.round(CDF,2),
                      textposition="middle left",marker=dict(size=20,symbol="line-ew-open")
                      ,marker_color="black",name="CDF")
    trace4=go.Scatter(x=x[1:i+1],y=CDF[1:i+1],mode='markers', hoverinfo="text",
                      textposition="middle left",marker=dict(size=9,symbol="x")
                      ,marker_color="blue",text="u>=cdf, move above",name="Failed comparisons")
    trace5=go.Scatter(x=x[[i+1]],y=CDF[[i+1]],mode='markers', hoverinfo="text",
                      textposition="middle left",marker=dict(size=9,symbol="circle")
                      ,name="Successful comparison",marker_color="blue",text="u<cdf, stop")

    g = go.FigureWidget(data=[trace1,trace2,trace3],
                        layout=go.Layout(
                            title=dict(
                            text=''
                            )
                            ,xaxis = dict(title = "Inverse Transform Method", range = [-0.5,0.5],showticklabels=False),
                            yaxis = dict(title = "", range = [-.1,1.1]),
                            hovermode=None,
                            margin={'l': 0, 'r': 0, 't': 0, 'b': 0},width=600, height=400 

                        ))

    return Outcomes_text,Probs_text, RNG, update, search_rn, RN_text, FB,g,trace4,trace5
Outcomes_text,Probs_text, RNG, update, search_rn, RN_text, FB,g,trace4,trace5=initialize()
 
Probs,CDF,x,Outcomes,RN=process_data(Probs_text, Outcomes_text, RN_text)
i=convert(Outcomes,CDF,RN) 
    
def response(change):
    Probs,CDF,x,Outcomes,RN=process_data(Probs_text, Outcomes_text, RN_text)
    if len(Probs)!=len(Outcomes):
        FB.value="<b>WARNING: # Outcomes and # Probs are different  </b>"
    elif np.isclose(np.sum(Probs),1)==0:
        print(np.sum(Probs))
        FB.value="<b>WARNING:Probs do not sum up to 1 </b>"
    elif np.min(Probs)<0:
        FB.value="<b>WARNING:Contains negative value</b>"
    elif len(np.unique(Outcomes))!=len(Probs):
        FB.value="<b>WARNING:Contains repetitive outcomes</b>"
    else:
        FB.value=""
        with g.batch_update():
            g.data[0].x = x
            g.data[0].y = Probs
            g.data[0].marker.color=CDF
            g.data[0].text=Outcomes
            g.data[1].y=[RN]
            i=convert(Outcomes,CDF,RN)  
            g.data[1].text="u="+str(RN)+"<br>converts to "+str(Outcomes[i])
            g.data[2].x = x
            g.data[2].y =CDF
            g.data[2].text=np.round(CDF,2)

            if search_rn.value==True:
                i=convert(Outcomes,CDF,RN)  
                g.data[3].x=x[1:i+1]
                g.data[3].y=CDF[1:i+1]
                g.data[4].x=x[[i+1]]
                g.data[4].y=CDF[[i+1]]


def response2(change):
    if search_rn.value==True:
        with g.batch_update():
            g.add_trace(trace4)
            g.add_trace(trace5)
            Probs,CDF,x,Outcomes,RN=process_data(Probs_text, Outcomes_text, RN_text)
            i=convert(Outcomes,CDF,RN)   
            g.data[3].x=x[1:i+1]
            g.data[3].y=CDF[1:i+1]
            g.data[4].x=x[[i+1]]
            g.data[4].y=CDF[[i+1]]
    else:
        g.data = [g.data[0], g.data[1],g.data[2]]



def change_random(b):
    RN_text.value=str(np.random.rand())

RNG.on_click(change_random )
update.on_click(response  )
search_rn.observe(response2,names="value")
RN_text.observe(response,names="value")
  
        
container1 = widgets.HBox([RNG,RN_text])
container2 = widgets.HBox([Outcomes_text,Probs_text,update])
container4 = widgets.HBox([search_rn])
container3 = widgets.HBox([FB])

 

Widget1=widgets.VBox([container1,container2,container3,container4, g])
Widget1
